In [2]:
import pandas as pd
from datetime import datetime
charging_session = pd.read_csv('charging_sessions2.csv')
bump = pd.read_csv('bump.csv')

In [3]:
bump.columns = bump.iloc[21]
bump = bump.iloc[22:-6,:]
bump.reset_index(drop=True, inplace=True)
bump['Date']= pd.to_datetime(bump['Date'].astype(str), infer_datetime_format=True)

In [4]:
charging_session['start_date']= pd.to_datetime(charging_session['start_date'].astype(str), infer_datetime_format=True)
charging_session['end_date']= pd.to_datetime(charging_session['end_date'].astype(str), infer_datetime_format=True)

In [5]:
charging_session.head()

,Unnamed: 0,plate_number,model,start_date,end_date,start_battery,end_battery,adress,location,driver,...,day,created_id,kWh,kWh_used,battery_used,last_mileage,last_plate,distance,consumption,site
0,1,FB-386-YK,ZOE,2022-03-30 11:30:13,2022-03-30 13:18:19,9.0,85.0,"15 Av. Charles de Gaulle, 92100 Boulogne-Billa...","48.85002136, 2.23768306",Van Huy NGUYEN,...,89,FB-386-YKVan Huy NGUYEN89,30.4,NaN,NaN,NaN,NaN,0.0,NaN,NO MATCH
1,7,FB-386-YK,ZOE,2022-04-11 20:19:00,2022-04-13 19:55:24,15.0,100.0,"5bis Rue Mahias, 92100 Boulogne-Billancourt, F...","48.84577179, 2.234308",Madil Berkani,...,101,FB-386-YKClient101,34.0,28.0,70.0,15939.0,FB-386-YK,163.0,0.171779,NO MATCH
2,9,FB-386-YK,ZOE,2022-04-18 16:39:24,2022-04-19 19:16:26,14.0,100.0,"12 Rue de lÉcole de Médecine, 75006 Paris, France","48.85112381, 2.34090495",luc talbot,...,108,FB-386-YKluc talbot108,34.4,34.4,86.0,16102.0,FB-386-YK,237.0,0.145148,SAEMES/SAE-EDM | 21 Rue de l'École de Médecine
3,10,FB-386-YK,ZOE,2022-04-24 19:18:29,2022-04-25 17:40:24,14.0,100.0,"40ter Rue Olivier Métra, 75020 Paris, France","48.8736496, 2.39352894",Daniel Da Silva Meneses Campos,...,114,FB-386-YKDaniel Da Silva Meneses Campos114,34.4,34.4,86.0,16339.0,FB-386-YK,240.0,0.143333,BMP - Paripark - Olivier Metra - Paris | 35-49...
4,17,FB-386-YK,ZOE,2022-05-09 16:52:00,2022-05-16 12:18:00,16.0,99.0,"79 Pl. Saint-Jacques, 75014 Paris, France","48.83380127, 2.3364501",Zity Paris 1,...,129,FB-386-YKZity Paris 1129,33.2,33.6,84.0,16579.0,FB-386-YK,205.0,0.163902,NO MATCH


In [6]:
bump.head()

21,NaN,N°,Date,RFID tag ID,Site,Energie (kWh),Durée (min),Prix Unitaire HT,Unité,Prix Total HT,Type,Borne
0,NaN,1,2022-05-22 18:34:07,0481BAF2116880,BMP - Campanile - Berges du Rhône - Lyon | 4-6...,24.80,96.60,€0.34,kWh,€8.43,DC,BER#3 - BMP - IES S/N000006
1,NaN,2,2022-05-16 12:26:43,0481BAF2116880,BMP - Campanile - Berges du Rhône - Lyon | 4-6...,20.90,74.08,€0.34,kWh,€7.11,DC,BER#2 - BMP - IES S/N000004
2,NaN,3,2022-05-16 15:58:29,0481BAF2116880,BMP - Campanile - Berges du Rhône - Lyon | 4-6...,24.50,96.95,€0.34,kWh,€8.33,DC,BER#2 - BMP - IES S/N000004
3,NaN,4,2022-05-17 12:34:05,0481BAF2116880,BMP - Campanile - Berges du Rhône - Lyon | 4-6...,24.20,95.32,€0.34,kWh,€8.23,DC,BER#2 - BMP - IES S/N000004
4,NaN,5,2022-05-17 15:58:30,0481BAF2116880,BMP - Campanile - Berges du Rhône - Lyon | 4-6...,25.40,94.05,€0.34,kWh,€8.64,DC,BER#2 - BMP - IES S/N000004


In [9]:
count = 0
bump['number_of_match'] = 0
bump['list_of_match'] = None
bump['list_of_match'] = bump['list_of_match'].astype('object')
for i in range(0,len(bump)):
    charging_position = []
    for j in range(0,len(charging_session)):
        if bump.loc[i,'Date'] >= charging_session.loc[j,'start_date'] and bump.loc[i,'Date'] <= charging_session.loc[j,'end_date'] and float(bump.loc[i,'Energie (kWh)']) >= float(charging_session.loc[j,'kWh'])-4 and float(bump.loc[i,'Energie (kWh)']) <= float(charging_session.loc[j,'kWh'])+4 and charging_session.loc[j,'site'] in bump.loc[i,'Site']:
            charging_position.append(j)
    bump.loc[i, 'number_of_match'] = len(charging_position)
    bump.at[i, 'list_of_match'] = charging_position    

KeyboardInterrupt: 

In [55]:
bump

Unnamed: 0                                                        NaN
N°                                                              298.0
Date                                              2022-04-23 08:53:39
RFID tag ID                                                  B4F2C8E3
Site                BMP - Paripark - Jardin des Plantes - Paris | ...
Energie (kWh)                                                    24.1
Durée (min)                                                     94.35
Prix Unitaire HT                                                €0.32
Unité                                                             kWh
Prix Total HT                                                   €7.71
Type                                                               DC
Day of year                                                     113.0
Hour                                                              8.0
Name: 297, dtype: object

In [56]:
charging.iloc[1215,:]

plate_number                                                         FB-070-YK
model                                                                      ZOE
start_date                                                 2022-04-23 07:46:21
end_date                                                   2022-04-23 10:54:09
start_battery                                                              0.0
end_battery                                                              100.0
adress                                     13 Rue Censier, 75005 Paris, France
driver                                                            Sseire Sylla
day                                                                      113.0
created_id                                           FB-070-YKSseire Sylla 113
distance                                                                 255.0
kWh_used                                                                  38.0
battery_used                                        

In [57]:
charging.loc[1215,'site'] in bump.loc[297,'Site']

True

In [60]:
bump.loc[297,'Date'] >= charging.loc[1215,'start_date'] and bump.loc[297,'Date'] <= charging.loc[1215,'end_date']

True

In [63]:
float(bump.loc[297,'Energie (kWh)']) >= float(charging.loc[1215,'kWh'])-4 and float(bump.loc[297,'Energie (kWh)']) <= float(charging.loc[1215,'kWh'])+4

True

In [64]:
if charging.loc[1215,'site'] in bump.loc[297,'Site'] and bump.loc[297,'Date'] >= charging.loc[1215,'start_date'] and bump.loc[297,'Date'] <= charging.loc[1215,'end_date'] and float(bump.loc[297,'Energie (kWh)']) >= float(charging.loc[1215,'kWh'])-4 and float(bump.loc[297,'Energie (kWh)']) <= float(charging.loc[1215,'kWh'])+4:
    print(1)

1
